In [4]:
!pip install asym_helper


ERROR: Could not find a version that satisfies the requirement asym_helper (from versions: none)
ERROR: No matching distribution found for asym_helper


## AEP

In [6]:
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import model_summary as model_class

%load_ext autoreload
%autoreload 2

In [8]:
df_aep = pd.read_csv("data/AEP_hourly.csv")
df_aep.rename(columns={'Datetime': 'ds', 'AEP_MW': 'y'}, inplace=True)
df_aep['ds'] = pd.to_datetime(df_aep['ds'], format='%Y-%m-%d %H:%M:%S')
out_dir = 'new_exp_results'

### test GRU model

In [9]:
train_config = model_class.training_config(n_epochs = 30, device = torch.device("mps"))
fourier_conf = model_class.fourier_config(mode="vector", K_weekly=3, K_monthly=6, K_yearly=10)
F_pairs = 2*(fourier_conf.K_weekly + fourier_conf.K_monthly + fourier_conf.K_yearly)
if fourier_conf.mode == "vector":
    cont_dim = 1 + F_pairs
    fourier_dim = F_pairs
else:
    cont_dim = 1 + 24*F_pairs
    fourier_dim = F_pairs

results_aep_fourier_w_matrix_gru = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y':[]})
date_start = pd.to_datetime('2004-10-01 00:00:00')
date_end = pd.to_datetime('2008-10-01 00:00:00')

i = 1

while True:
    print(i)
    df_reduced = df_aep.loc[(df_aep['ds'] >= date_start) & (df_aep['ds'] < date_end)].copy()
    model = model_class.RNN_fourier_GRU(
        cont_dim=cont_dim,     # full feature dim per step
        fourier_dim=F_pairs, # only used in hand-rolled branch
        xf_mode="vector",           # or "matrix" to match your original
        d_model=128,
        nhead=4,
        H=24,
        use_gate=True,
        nonneg_U0=False,
        use_gru=True,               # turn on GRU backbone (process all x together)
    )
    trainer = model_class.RNN_train_fourier(model, train_config, fourier_conf)
    forcast, true = trainer(df_reduced)

    result = pd.DataFrame({'day': df_reduced['ds'].dt.date.max(), 'hour': list(range(24)), 'y_hat': forcast, 'y': true})

    results_aep_fourier_w_matrix_gru = pd.concat([results_aep_fourier_w_matrix_gru, result]).reset_index(drop=True)

    date_start += pd.Timedelta(days=1)
    date_end += pd.Timedelta(days=1)
    if i == 70:
        print('finish')
        break
    else:
        i += 1

results_aep_fourier_w_matrix_gru.to_csv(out_dir + '/results_aep_fourier_gru_kw3_km6_ky10_matrix.csv', index=False)

1


/Users/zero/Documents/GitHub/RNN_based_downscaling/model_summary.py:1484: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:836.)
  print(f"epoch {ep + 1} loss: {float(loss):.4f}")


epoch 1 loss: 0.7645
epoch 2 loss: 0.4159
epoch 3 loss: 0.3810
epoch 4 loss: 0.2755
epoch 5 loss: 0.1947
epoch 6 loss: 0.1579
epoch 7 loss: 0.1267
epoch 8 loss: 0.1151
epoch 9 loss: 0.1068
epoch 10 loss: 0.0954
epoch 11 loss: 0.0772
epoch 12 loss: 0.0794
epoch 13 loss: 0.0705
epoch 14 loss: 0.0748
epoch 15 loss: 0.0687
epoch 16 loss: 0.0692
epoch 17 loss: 0.0643
epoch 18 loss: 0.0607
epoch 19 loss: 0.0621
epoch 20 loss: 0.0529
epoch 21 loss: 0.0527
epoch 22 loss: 0.0566
epoch 23 loss: 0.0597
epoch 24 loss: 0.0520
epoch 25 loss: 0.0485
epoch 26 loss: 0.0506
epoch 27 loss: 0.0450
epoch 28 loss: 0.0392
epoch 29 loss: 0.0382
epoch 30 loss: 0.0399


/var/folders/3h/5yfj7yqx7lg98t310bfxxr3m0000gn/T/ipykernel_48333/131365095.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_aep_fourier_w_matrix_gru = pd.concat([results_aep_fourier_w_matrix_gru, result]).reset_index(drop=True)


2


KeyboardInterrupt: 

In [ ]:
import model_dich_mu_std_laa

In [12]:

class Cfg:
    pass

cfg = Cfg()
cfg.device = "mps"            # 或 "cuda"/"cpu"
cfg.n_epochs = 90
cfg.lr = 5e-4
cfg.x_col = "x"
cfg.y_cols = [f"y_{i}" for i in range(24)]
cfg.T_hist = 32
cfg.batch_size = 64
cfg.test_ratio = 0.2
cfg.kl_coeff = 1.0            # 新版loss不使用KL, 留着也无妨
cfg.pred_samples = 100000

# ---- 结果容器 ----
results_prob_dich = pd.DataFrame({'day': [], 'hour': [], 'y_hat': [], 'y': []})

# ---- 滑动时间窗 ----
date_start = pd.to_datetime('2004-10-01 00:00:00')
date_end   = pd.to_datetime('2008-10-01 00:00:00')

i = 1
while True:
    print(i)
    # 取一天/一段窗口的数据（按你提供的逻辑）
    df_reduced = df_aep.loc[(df_aep['ds'] >= date_start) & (df_aep['ds'] < date_end)].copy()

    # ---- 建模：K=24（份额维度），d_x=1（x是总量一列） ----
    model = model_dich_mu_std_laa.DirichletMeanConcentration(
        K=24,            # 24 小时
        d_model=128,     # 可调
        d_x=11,           # 外生/总量特征维度
        nhead=4,
        dropout=0.1,
        activation="relu",
        tie_c_to_h_only=True
    )

    trainer = model_dich_mu_std_laa.RNN_train(model, cfg, train_sample = False)
    ret = trainer(df_reduced)

    # 取当天的 y_hat 与真实 y（注意转成 numpy）
    y_hat = ret['Y_pred'].flatten().cpu().numpy()
    y_true = ret['Y_test'].flatten().cpu().numpy()

    day_val = pd.to_datetime(df_reduced['ds'].max()).date()
    result = pd.DataFrame({
        'day':  [day_val] * 24,
        'hour': list(range(24)),
        'y_hat': y_hat,
        'y':    y_true
    })

    results_prob_dich = pd.concat([results_prob_dich, result], ignore_index=True)

    # 窗口右移一天
    date_start += pd.Timedelta(days=1)
    date_end   += pd.Timedelta(days=1)

    if i == 3:
        print('finish')
        break
    else:
        i += 1

# 保存结果
results_prob_dich.to_csv(out_dir + '/model_dich_mu_std_lla_pw7.csv', index=False)


1


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x39 and 15x24)